In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [2]:
path = '/media/onno/Algemeen/Thesis/GFS_T850/duration_error/'

Best and worst annual forecasts

In [11]:
path_to_write = '/media/onno/Algemeen/Thesis/GFS_T850/rank_forecast/duration/'
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
#Good forecasts
columnz = ['persistent_hw','persistent_cw','short_hw','short_cw']
modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
lead_dayz = [3,5]
file = 'duration_error_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for model in modelz:
        for lead_day in lead_dayz:
            for event in columnz:
                for season in seasonz:
                    df = pd.read_csv(path+file.format(event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),
                                     index_col=0)
                    df_good = df.where((df['duration_error']>=-1) & (df['duration_error']<=1)).dropna()
                    df_bad = df.where(df['duration_error']<-1).dropna()
                    df_good.to_csv(path_to_write+'good_forecast_'+file.format(event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day))
                    df_bad.to_csv(path_to_write+'bad_forecast_'+file.format(event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day))
            
    
    

In [17]:
path_to_write = '/media/onno/Algemeen/Thesis/GFS_T850/rank_forecast/'
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
#Good forecasts
columnz = ['persistent_hw','persistent_cw','short_hw','short_cw']
modelz = ['GFS','ERA5RF']
lead_day = 3
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    df_MAE_GFS = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_{}_lon_{}_{}_lat_{}_{}_GFS.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_MAE_GFS.index = pd.to_datetime(df_MAE_GFS.index)
    df_MAE_ERA5RF = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_{}_lon_{}_{}_lat_{}_{}_ERA5RF.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_MAE_ERA5RF.index = pd.to_datetime(df_MAE_ERA5RF.index)
    df_GSS_GFS = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_{}_lon_{}_{}_lat_{}_{}_GFS.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_GSS_GFS.index = pd.to_datetime(df_GSS_GFS.index)
    df_GSS_ERA5RF = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_{}_lon_{}_{}_lat_{}_{}_ERA5RF.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_GSS_ERA5RF.index = pd.to_datetime(df_GSS_ERA5RF.index)
    for column in columnz:
        if (column=='persistent_hw') | (column=='persistent_cw'):
            limit=10
        else:
            limit=30
        for model in modelz:
            if model == 'ERA5RF':
                column_MAE = column+'_MAE'
                df_good = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                df_good[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
                # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                df_good = df_good.sort_values(column+'_MAE').where(df_good[column+'_GSS']>0.6).dropna()
                
                df_good[:limit].to_csv(path_to_write+'good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}.txt'.format(column,lon_0,lon_1,lat_0,lat_1,model))

                #Bad forecasts
                column_MAE = column+'_MAE'
                df_bad = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                df_bad[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
                # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                df_bad = df_bad.sort_values(column+'_MAE',ascending=False).where(df_bad[column+'_GSS']<0.2).dropna()
                df_bad[:limit].to_csv(path_to_write+'bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}.txt'.format(column,lon_0,lon_1,lat_0,lat_1,model))
            else:
                column_MAE = column+'_MAE'
                df_good = df_MAE_GFS.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                df_good[column+'_GSS']=df_GSS_GFS.loc[:,[column]].dropna()
                # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                df_good = df_good.sort_values(column+'_MAE').where(df_good[column+'_GSS']>0.6).dropna()
                df_good[:limit].to_csv(path_to_write+'good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}.txt'.format(column,lon_0,lon_1,lat_0,lat_1,model))

                #Bad forecasts
                column_MAE = column+'_MAE'
                df_bad = df_MAE_GFS.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                df_bad[column+'_GSS']=df_GSS_GFS.loc[:,[column]].dropna()
                # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                df_bad = df_bad.sort_values(column+'_MAE',ascending=False).where(df_bad[column+'_GSS']<0.2).dropna()
                df_bad[:limit].to_csv(path_to_write+'bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}.txt'.format(column,lon_0,lon_1,lat_0,lat_1,model))

Best and worst half forecasts

In [5]:
path_to_write = '/media/onno/Algemeen/Thesis/GFS_T850/rank_forecast/'
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
columnz = ['persistent_hw','persistent_cw','short_hw','short_cw']
limitz = [5,5,15,15]

seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

lead_day = 5
modelz = ['GFS',"ERA5RF"]
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    df_MAE_GFS = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_{}_lon_{}_{}_lat_{}_{}_GFS.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_MAE_GFS.index = pd.to_datetime(df_MAE_GFS.index)
    df_MAE_ERA5RF = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_{}_lon_{}_{}_lat_{}_{}_ERA5RF.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_MAE_ERA5RF.index = pd.to_datetime(df_MAE_ERA5RF.index)
    df_GSS_GFS = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_{}_lon_{}_{}_lat_{}_{}_GFS.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_GSS_GFS.index = pd.to_datetime(df_GSS_GFS.index)
    df_GSS_ERA5RF = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_{}_lon_{}_{}_lat_{}_{}_ERA5RF.csv'.format(lead_day,lon_0,lon_1,lat_0,lat_1),index_col=0)
    df_GSS_ERA5RF.index = pd.to_datetime(df_GSS_ERA5RF.index)
    for i,column in enumerate(columnz):
        for season in seasonz:
            for model in modelz:
                if model == 'ERA5RF':
                    column_MAE = column+'_MAE'
                    df_good = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                    df_good[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
                    # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                    df_good = df_good.sort_values(column+'_MAE')#.where(df_good[column+'_GSS']>0.6).dropna()
                    df_good = df_good[np.isin(df_good.index.month,seasonz[season])]
                    df_good[:len(df_good)//2].to_csv(path_to_write+'good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(column,lon_0,lon_1,
                                                                                                          lat_0,lat_1,season,model,lead_day))

                    #Bad forecasts
                    column_MAE = column+'_MAE'
                    df_bad = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                    df_bad[column+'_GSS']=df_GSS_ERA5RF.loc[:,[column]].dropna()
                    # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                    df_bad = df_bad.sort_values(column+'_MAE',ascending=False)#.where(df_bad[column+'_GSS']<0.2).dropna()
                    df_bad = df_bad[np.isin(df_bad.index.month,seasonz[season])]
                    df_bad[:len(df_good)//2].to_csv(path_to_write+'bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(column,lon_0,lon_1,
                                                                                                      lat_0,lat_1,season,model,lead_day))
                else:
                    column_MAE = column+'_MAE'
                    df_good = df_MAE_GFS.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                    df_good[column+'_GSS']=df_GSS_GFS.loc[:,[column]].dropna()
                    # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                    df_good = df_good.sort_values(column+'_MAE')#.where(df_good[column+'_GSS']>0.6).dropna()
                    df_good = df_good[np.isin(df_good.index.month,seasonz[season])]
                    df_good[:len(df_good)//2].to_csv(path_to_write+'good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(column,lon_0,lon_1,
                                                                                                           lat_0,lat_1,season,model,lead_day))
                    #Bad forecasts
                    column_MAE = column+'_MAE'
                    df_bad = df_MAE_GFS.loc[:,[column]].dropna().rename(columns={column:column_MAE})
                    df_bad[column+'_GSS']=df_GSS_GFS.loc[:,[column]].dropna()
                    # df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
                    df_bad = df_bad.sort_values(column+'_MAE',ascending=False)#.where(df_bad[column+'_GSS']<0.2).dropna()
                    df_bad = df_bad[np.isin(df_bad.index.month,seasonz[season])]
                    df_bad[:len(df_good)//2].to_csv(path_to_write+'bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(column,lon_0,lon_1,
                                                                                                      lat_0,lat_1,season,model,lead_day))        
